# Numerical Analysis - IMPA 2020
### Professor Dan Marchesin
### Hallison Paz, 1st year Phd student
### Pedro Fonini, 2nd year MSc student

# Modeling epidemics with SIR model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual

In [ ]:
import os
OUTPUT_DIR = os.path.join('images', 'SIR')

## Solving SIR model using Euler Method

In [ ]:
def SIR(I0=0.0001,
        dt = 1,
        days = 20,
        beta=2.5, 
        gamma=0.8,
        corrector=False):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)

    S = np.zeros(n_samples)
    I = np.zeros(n_samples)
    R = np.zeros(n_samples)
    # Initial condition
    R[0] = 0
    I[0] = I0
    S[0] = 1 - I0

    # Step equations forward in time
    for n in range(len(t)-1):
        # predictor
        S[n+1] = S[n] - dt*beta*S[n]*I[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n] 
        R[n+1] = R[n] + dt*gamma*I[n]
        if corrector:
            S[n+1] = S[n] - dt*beta*S[n+1]*I[n+1]
            I[n+1] = I[n] + dt*beta*S[n+1]*I[n+1] - dt*gamma*I[n+1] 
            R[n+1] = R[n] + dt*gamma*I[n+1]
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S, I, R

In [ ]:
# Initial condition (S0 and R0 are determided by I0)
I0 = 0.1
# Parameters
dt = 1
days = 20
beta = 2.5
gamma = 0.8 
# simulations = {}
S, I, R = SIR(I0, dt, days, beta, gamma)

## Plotting Curves for multiple values of Beta and Gamma

In [ ]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2))
def draw_SIR_curves(I0 = 0.11,
                    dt = 1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    corrector = False):
    S, I, R = SIR(I0, dt, days, beta, gamma, corrector)
    t = [dt*i for i in range(len(S))]
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S', 'I', 'R'), 'center right')
    plt.xlabel('Time')
    plt.ylabel('% population')
    plt.show()

## Ploting orbits in the triangle S + I <= 1 (Autonomous system of ODE)

In [ ]:
@interact_manual(samples=(2, 50, 1), 
                 dt=(0.1, 2),
                days=20)
def draw_SI_relation(samples = 11,
                    dt = 0.1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    corrector = False):
    s_list = []
    i_list = []
    for i0 in np.linspace(0, 1.0, samples):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        s_list.append(S)
        i_list.append(I)
        

    fig = plt.figure(figsize=(10, 10))
    plt.plot([0.0, 1.0], [1.0, 0.0], 'gray')
    for k in range(len(s_list)):
        plt.plot(s_list[k], i_list[k], 'blue' if k%2 else '#0000ff80')
        
    for i0 in np.linspace(0.0, 0.1, int(samples/2)):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        plt.plot(S, I, 'red' if k%2 else '#ff000080')
        
    plt.xlabel('Susceptible')
    plt.ylabel('Infected')
    plt.show()

## Comparing Euler Method with and without Predictor-Corrector technique

In [ ]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2),
                 show=['all', 'S', 'I', 'R'])
def compare_methods(I0 = 0.11,
                    dt = 1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    show='all'):
    
    fe_sir = SIR(I0, dt, days, beta, gamma, False)
    pc_sir = SIR(I0, dt, days, beta, gamma, True)
    t =  [dt*i for i in range(len(fe_sir[0]))]
    legends =['S', 'I', 'R']
    
    fig, ax = plt.subplots(1, 3, figsize=(28,8)) if show == 'all' else (plt.figure(figsize=(16,8)), None)
    if show == 'all':
        for index in range(len(fe_sir)):
            ax[index].plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
            ax[index].plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
            ax[index].legend(loc= 'center right')
    else:
        index = legends.index(show)
        plt.plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
        plt.plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
        fig.legend(loc= 'center right')
        
    fig.suptitle('Comparinson between methods for ODE')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.show()
    